<a href="https://colab.research.google.com/github/sebasas15/viz_covid/blob/master/viz_covid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
usar pydeck con datos de https://github.com/MinCiencia/Datos-COVID19 para mostrar evolucion de covid
usar google trends para conseguir lo mas visto en las comunas mas afectadas
 

SyntaxError: ignored

In [23]:
!pip install pydeck
!pip install SPARQLWrapper
!pip install unidecode
!pip install paho-mqtt


In [0]:
#CONSTANTS 
 MAPBOX_API_KEY="pk.eyJ1Ijoic2ViYXNhczE1IiwiYSI6ImNrOXcyNXAwdjAycWMzbHA3cDBic2k0MHIifQ.8lNIY2s_y-V82-Si_x7YbQ"

###Conseguir data de infectados en chile con PANDAS

In [0]:
import pandas as pd
from datetime import datetime,timedelta
from pytz import timezone

day= datetime.now(tz=timezone('America/Santiago'))
c=0
max_attempts=4
while 1:
  INFECTADOS_CHILE=f'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto2/{day.strftime("%Y-%m-%d")}-CasosConfirmados.csv'
  try:
    data = pd.read_csv(INFECTADOS_CHILE)
    break
  except:
    day = day -  timedelta(days=1)
    if c>=max_attempts: 
      break
    c=c+1






###Conseguir datos de geolocalizacion de comunas con SPARQL

In [0]:

import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

query = """
SELECT (SAMPLE(?location) as ?location) (SAMPLE(?label_es) AS ?label_es) ?comuna
WHERE 
{
  VALUES ?regionMetropolitana { wd:Q2131 } 
  
  ?regionMetropolitana wdt:P150 ?provincia.
  ?provincia wdt:P150 ?comuna.
  FILTER NOT EXISTS { ?provincia pq:P582 ?x }
  #consigo el nombre de la comuna
  OPTIONAL { ?comuna rdfs:label ?label_es . FILTER(LANG(?label_es) = "es") }
  #consigo las coordenadas
  OPTIONAL { ?comuna wdt:P625 ?location }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],es". }
}
#elimino duplicados
GROUP BY ?comuna
"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


sparql_results = get_results(endpoint_url, query)

#un mapa localizacion, nombre



In [0]:
import unidecode
import re

def mapping(elem):
  label =  unidecode.unidecode(elem["label_es"]["value"])
  latitude = float(re.search(r'\((.*)\s', elem["location"]["value"])[1])
  longitude = float( re.search(r'\s(.*)\)', elem["location"]["value"])[1])
  return (label,[latitude,longitude])


region_location = dict(map(mapping , sparql_results["results"]["bindings"]))



filtered = data[data["Region"]=="Metropolitana"]
#matriz de puntos para insertar en pydeck
data_points = pd.DataFrame(columns=['lng','lat'])


for item in filtered.iterrows():
  casos = item[1]["Casos Confirmados"]
  latlong = region_location.get(item[1]["Comuna"])
  multi=list()
  for i in range(0, int(casos)):
    if latlong != None:
      multi.append(latlong)
  tmpDataFrame = pd.DataFrame(multi, columns=['lng','lat'])
  data_points = data_points.append(tmpDataFrame)



In [0]:
data_points.head(3)

,lng,lat
0,-70.75,-33.733333
1,-70.75,-33.733333
2,-70.75,-33.733333


In [28]:
import pydeck
# 2014 locations of car accidents in the UK
UK_ACCIDENTS_DATA = ('https://raw.githubusercontent.com/uber-common/'
                     'deck.gl-data/master/examples/3d-heatmap/heatmap-data.csv')

# Define a layer to display on a map
layer = pydeck.Layer(
  
    'HeatmapLayer',
    data_points,
    get_position='[lng, lat]',
    auto_highlight=True,
    elevation_scale=10,
    pickable=True,
    elevation_range=[0, 3000],
    extruded=True,                 
    coverage=1)


# Set the viewport location
view_state = pydeck.ViewState(
    longitude=-70.669266,
    latitude=-33.448891,
    zoom=8,
    min_zoom=5,
    max_zoom=15,
    pitch=40.5,
    bearing=-27.36)

# Render
r = pydeck.Deck(layers=[layer], initial_view_state=view_state,mapbox_key=MAPBOX_API_KEY)
#r.show()
r.to_html('demo.html',)


<IPython.core.display.Javascript object>

'/content/demo.html'

### Interactivo con mqtt

In [0]:
import paho.mqtt.client as mqtt
def on_connect(client, userdata, flags, rc):
    print("Connected with result code "+str(rc))

    # Subscribing in on_connect() means that if we lose the connection and
    # reconnect then subscriptions will be renewed.
    client.subscribe("test/jupyter/1")

# The callback for when a PUBLISH message is received from the server.
def on_message(client, userdata, msg):
    if "test/jupyter/1" == msg.topic:
      print(msg.topic+" "+str(msg.payload))

client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

client.connect("mqtt.eclipse.org", 1883, 60)
client.loop_forever()
